<a href="https://colab.research.google.com/github/distractedm1nd/mar2moon/blob/main/scripts/Wav2Vec2_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets==1.4.1
!pip install transformers==4.4.0
!pip install soundfile
!pip install jiwer
!pip install huggingface_hub

     |████████████████████████████████| 194kB 15.5MB/s 
     |████████████████████████████████| 245kB 43.6MB/s 
     |████████████████████████████████| 122kB 54.5MB/s 
     |████████████████████████████████| 2.1MB 14.9MB/s 
     |████████████████████████████████| 3.3MB 31.3MB/s 
     |████████████████████████████████| 901kB 45.6MB/s 
     |████████████████████████████████| 51kB 5.6MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149809 sha256=6a4b3a81ea78328e66f0760bca9e09f1afdd54e2c5fc830b766de95038956c61
  Stored in directory: /root/.cache/pip/wheels/b3/26/73/4b48503bac73f01cf18e52cd250947049a7f339e940c5df8fc
Successfully built python-Levenshtein


In [2]:
!nvidia-smi

Sat Jul  3 05:15:34 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P0    27W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [25]:
from datasets import Dataset
from IPython.display import display, HTML
import pandas as pd
import numpy as np
import re
import os
import random

df = pd.read_csv("drive/MyDrive/data4.csv")
df = df.dropna(subset=["Podcast_Title"])[["Podcast_Title", "corrected"]]

# drop all transcriptions containing numbers, because our processor doesnt handle them
df = df[~df.corrected.str.contains(r'[0-9]')]

# Only use transcriptions for the clips I have downloaded 
df["file"] = df.apply(lambda x: f"drive/MyDrive/clips/3/{x['Podcast_Title'][:-7]}_{x.name}.wav" if f"{x['Podcast_Title'][:-7]}_{x.name}.wav" in os.listdir("drive/MyDrive/clips/3") else np.nan, axis=1)
batch = Dataset.from_pandas(df.dropna(subset=["file"])).shuffle().train_test_split(test_size=0.3)
df.dropna(subset=["file"])

,Podcast_Title,corrected,file
0,Altcoin Daily_20210401_ETH to CRUSH Bitcoin (T...,could outperform bitcoin and is watching it ve...,drive/MyDrive/clips/3/Altcoin Daily_20210401_E...
1,Altcoin Daily_20210401_ETH to CRUSH Bitcoin (T...,has a slightly more nuanced opinion than many ...,drive/MyDrive/clips/3/Altcoin Daily_20210401_E...
2,Altcoin Daily_20210401_ETH to CRUSH Bitcoin (T...,to be more um sound from a monetary perspectiv...,drive/MyDrive/clips/3/Altcoin Daily_20210401_E...
3,Altcoin Daily_20210401_ETH to CRUSH Bitcoin (T...,so in bitcoin hard forks are extremely rare in...,drive/MyDrive/clips/3/Altcoin Daily_20210401_E...
4,Altcoin Daily_20210401_ETH to CRUSH Bitcoin (T...,the market but nevertheless the minority a hun...,drive/MyDrive/clips/3/Altcoin Daily_20210401_E...
...,...,...,...
1581,Crypto_Daily_Trade_Signals_20210518_DOGECOIN_S...,at one point five billion in dogecoin that uh ...,drive/MyDrive/clips/3/Crypto_Daily_Trade_Signa...
1582,Crypto_Daily_Trade_Signals_20210518_DOGECOIN_S...,ut things do point to the fact that this is go...,drive/MyDrive/clips/3/Crypto_Daily_Trade_Signa...
1583,Crypto_Daily_Trade_Signals_20210518_DOGECOIN_S...,have confirmation this is actually going to ha...,drive/MyDrive/clips/3/Crypto_Daily_Trade_Signa...
1584,Crypto_Daily_Trade_Signals_20210518_DOGECOIN_S...,own below so you can actually read that articl...,drive/MyDrive/clips/3/Crypto_Daily_Trade_Signa...


In [26]:
# Remove characters that we don't map
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\d\[\]]'

def remove_special_characters(batch):
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["corrected"]).upper()
    return batch

batch = batch.map(remove_special_characters)

def extract_all_chars(batch):
    all_text = " ".join(batch["text"])
    vocab = list(set(all_text))
    return {"vocab": [vocab], "all_text": [all_text]}

vocabs = batch.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=batch.column_names["train"])

# Create a character dict for our processor
vocab_list = list(set(vocabs["train"]["vocab"][0]) | set(vocabs["test"]["vocab"][0]))
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{' ': 12,
 "'": 27,
 'A': 7,
 'B': 20,
 'C': 24,
 'D': 10,
 'E': 19,
 'F': 0,
 'G': 11,
 'H': 23,
 'I': 26,
 'J': 16,
 'K': 25,
 'L': 2,
 'M': 1,
 'N': 17,
 'O': 13,
 'P': 4,
 'Q': 15,
 'R': 22,
 'S': 14,
 'T': 8,
 'U': 6,
 'V': 18,
 'W': 5,
 'X': 9,
 'Y': 21,
 'Z': 3}

In [27]:
# | is our separation token instead of a space
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

# add unknown and padding tokens for CTC
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
print(len(vocab_dict))

30


In [28]:
import json

# save to disk
with open('vocab2.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [29]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab2.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [30]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [31]:
from transformers import Wav2Vec2Processor

#processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h")
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [32]:
import soundfile as sf

# Load audio
def speech_file_to_array_fn(batch):
    global total_audiolength
    speech_array, sampling_rate = sf.read(batch["file"])
    total_audiolength += len(speech_array)
    batch["speech"] = speech_array
    # Sampling rate of the audio must also be set
    batch["sampling_rate"] = sampling_rate
    batch["target_text"] = batch["text"]
    return batch

batch = batch.map(speech_file_to_array_fn, remove_columns=batch.column_names["train"], num_proc=4)
print(total_audiolength)





344320


In [33]:
import IPython.display as ipd
import numpy as np

# Test if we load our audio data correctly
print(batch["train"][5]["target_text"])
ipd.Audio(data=np.asarray(batch["train"][5]["speech"]), autoplay=False, rate=16000)

OTES AND ETHEREUM TRADED NOTES ETC GROUP HAVE GOT PHYSICAL BITCOIN AND PHYSICAL ETHEREUM AND AGAIN THESE HAVE ALL GOT THEIR OWN ASSOCIATED MANAGEMENT FEES ASSOCIATED WITH THEM BUT AT LEAST


In [12]:
def prepare_dataset(batch):
    # check that all files have the correct sampling rate
    assert (
        len(set(batch["sampling_rate"])) == 1
    ), f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."

    batch["input_values"] = processor(batch["speech"], sampling_rate=batch["sampling_rate"][0]).input_values

    # Use our processor to generate the input ids from the audio tensors
    with processor.as_target_processor():
        batch["labels"] = processor(batch["target_text"]).input_ids
    return batch

# Batch size 8 works pretty well
batch_prepared = batch.map(prepare_dataset, remove_columns=batch.column_names["train"], num_proc=4, batched=True, batch_size=8)

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [34]:
import torch
import numpy as np
from datasets import load_metric
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)
wer_metric = load_metric("wer")

def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [45]:
from transformers import Wav2Vec2ForCTC

# Load pretrained model to fine tune, use gradient checkpointing and loss reduction to save memory
model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-960h",
    gradient_checkpointing=True,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
)
model.freeze_feature_extractor()

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [46]:
# Translate model output to text that we can use
def map_to_result(batch):
    model.to("cuda")
    input_values = processor(
        batch["speech"],
        sampling_rate=batch["sampling_rate"],
        return_tensors="pt"
    ).input_values.to("cuda")

    with torch.no_grad():
        logits = model(input_values).logits

    pred_ids = torch.argmax(logits, dim=-1)
    batch["pred_str"] = processor.batch_decode(pred_ids)[0]

    return batch

# Test WER before training
results = batch["test"].map(map_to_result)
print("WER Before Training: {:.3f}".format(wer_metric.compute(predictions=results["pred_str"], references=results["target_text"])))


WER Before Training: 0.270


In [48]:
from transformers import Trainer
from transformers import TrainingArguments

# Set model to training mode
model.train()

training_args = TrainingArguments(
  output_dir="distractedm1nd/wav2vec-en-finetuned-on-cryptocurrency",
  group_by_length=True,
  per_device_train_batch_size=4,
  evaluation_strategy="steps",
  num_train_epochs=25,
  fp16=False,
  learning_rate=1e-5,
  logging_first_step=True,
  weight_decay=0.005,
  save_steps=100,
  eval_steps=100,
  logging_steps=50,
  warmup_steps=600,
  save_total_limit=4,
)

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=batch_prepared["train"],
    eval_dataset=batch_prepared["test"],
    tokenizer=processor.feature_extractor,
)

In [49]:
# Start training!
trainer.train()

Step,Training Loss,Validation Loss,Wer,Runtime,Samples Per Second
100,0.894600,0.608472,0.274723,34.520900,4.896000
200,0.815000,0.555315,0.264259,35.007300,4.828000
300,0.662500,0.520206,0.249077,35.499400,4.761000
400,0.588200,0.453579,0.235741,34.968700,4.833000
500,0.572100,0.439139,0.223225,35.462900,4.766000
600,0.507900,0.398983,0.218506,35.205800,4.800000
700,0.472900,0.397481,0.209069,35.139700,4.809000
800,0.445700,0.407386,0.209069,35.904500,4.707000
900,0.406300,0.413111,0.200246,35.819200,4.718000
1000,0.402800,0.415675,0.197784,35.923400,4.704000


TrainOutput(global_step=4225, training_loss=0.3567828175019936, metrics={'train_runtime': 7753.6856, 'train_samples_per_second': 0.545, 'total_flos': 5.160626274072483e+18, 'epoch': 25.0, 'init_mem_cpu_alloc_delta': 49752, 'init_mem_gpu_alloc_delta': 0, 'init_mem_cpu_peaked_delta': 18306, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 1759831, 'train_mem_gpu_alloc_delta': 3742711808, 'train_mem_cpu_peaked_delta': 67668002, 'train_mem_gpu_peaked_delta': 2273230336})

In [1]:
# Set model to evaluation mode (turn off regularization)
model.eval()

results = batch["test"].map(map_to_result)
print("WER After Training: {:.3f}".format(wer_metric.compute(predictions=results["pred_str"], references=results["target_text"])))

NameError: ignored